In [1]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import lifelines
from mice_dfi import plots

## Load data

In [2]:
df_data_mpd = pd.read_csv('./generated/mpd_data.csv')
df_data_mpd = df_data_mpd.dropna(subset=['mort_age'])
cbc_features = [ 'gr %', 'gr (k/ul)', 'hb (g/dl)', 'hct %',
       'ly %', 'ly (k/ul)', 'mch (pg)', 'mchc (g/dl)', 'mcv(fl)', 'plt (k/ul)',
       'rbc (m/ul)', 'wbc (k/ul)']

## Train Cox PH model on CBC data

In [3]:
cox = lifelines.CoxPHFitter()
df_ = df_data_mpd.copy()
df_.loc[df_.sex=='M', 'sex_int'] = 1
df_.loc[df_.sex=='F', 'sex_int'] = 0
df_ = df_.dropna(subset=cbc_features + ['mort_tte', 'mort_event'])

# Cox PH model including age and sex
cox.fit(df_[['age', 'sex_int', 'mort_tte', 'mort_event'] + cbc_features], 'mort_tte', 'mort_event')
cox.print_summary(style=None)
betas = np.array(cox.summary['coef'].loc[cbc_features])
df_data_mpd['Cox PH'] = np.dot(df_data_mpd[cbc_features], betas)

# Cox PH model including sex
cox.fit(df_[['sex_int', 'mort_tte', 'mort_event'] + cbc_features], 'mort_tte', 'mort_event')
cox.print_summary(style=None)
betas = np.array(cox.summary['coef'].loc[cbc_features])
df_data_mpd['Cox PH (no age)'] = np.dot(df_data_mpd[cbc_features], betas)

<lifelines.CoxPHFitter: fitted with 1066 total observations, 96 right-censored observations>
             duration col = 'mort_tte'
                event col = 'mort_event'
      baseline estimation = breslow
   number of observations = 1066
number of events observed = 970
   partial log-likelihood = -5444.79
         time fit was run = 2022-08-14 12:33:19 UTC

---
              coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                           
age           0.03       1.03       0.00             0.03             0.04                 1.03                 1.04
sex_int      -0.27       0.76       0.07            -0.42            -0.12                 0.66                 0.88
gr %          0.10       1.11       0.05            -0.01             0.21                 0.99                 1.23
gr (k/ul)    -0.60       0.55       0.61            -1.79             0.60                 0.17                 1.82
hb (g/dl)     0.11       1.12       0.38            -0.64             0.87                 0.53                 2.38
hct %        -0.02       0.98       0.14            -0.31             0.26                 0.74                 1.29
ly %          0.08       1.08       0.05            -0.03             0.18                 0.97                 1.20
ly (k/ul)    -0.51       0.60       0.59            -1.68             0.65                 0.19                 1.91
mch (pg)     -0.25       0.78       0.39            -1.01             0.52                 0.36                 1.69
mchc (g/dl)  -0.01       0.99       0.03            -0.08             0.05                 0.92                 1.05
mcv(fl)       0.04       1.04       0.14            -0.24             0.32                 0.78                 1.38
plt (k/ul)   -0.00       1.00       0.00            -0.00            -0.00                 1.00                 1.00
rbc (m/ul)   -0.40       0.67       0.47            -1.31             0.51                 0.27                 1.67
wbc (k/ul)    0.52       1.69       0.58            -0.61             1.66                 0.54                 5.28

              cmp to     z      p   -log2(p)
covariate                                   
age             0.00 16.96 <0.005     212.03
sex_int         0.00 -3.64 <0.005      11.82
gr %            0.00  1.86   0.06       3.98
gr (k/ul)       0.00 -0.98   0.33       1.61
hb (g/dl)       0.00  0.29   0.77       0.38
hct %           0.00 -0.17   0.86       0.21
ly %            0.00  1.42   0.16       2.69
ly (k/ul)       0.00 -0.87   0.39       1.37
mch (pg)        0.00 -0.63   0.53       0.92
mchc (g/dl)     0.00 -0.39   0.70       0.52
mcv(fl)         0.00  0.26   0.79       0.33
plt (k/ul)      0.00 -2.93 <0.005       8.21
rbc (m/ul)      0.00 -0.86   0.39       1.36
wbc (k/ul)      0.00  0.90   0.37       1.45
---
Concordance = 0.75
Partial AIC = 10917.57
log-likelihood ratio test = 620.96 on 14 df
-log2(p) of ll-ratio test = 407.72

<lifelines.CoxPHFitter: fitted with 1066 total observations, 96 right-censored observations>
             duration col = 'mort_tte'
                event col = 'mort_event'
      baseline estimation = breslow
   number of observations = 1066
number of events observed = 970
   partial log-likelihood = -5588.72
         time fit was run = 2022-08-14 12:33:19 UTC

---
              coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                           
sex_int      -0.44       0.64       0.07            -0.58            -0.29                 0.56                 0.75
gr %          0.02       1.02       0.05            -0.09             0.12                 0.92                 1.13
gr (k/ul)    -0.56       0.57       0.60            -1.74             0.62                 0.18                 1.86
hb (g/dl)    -0.78       0.46       0.36            -1.48            -0.07                 0.23                 0.93
hct %         0.18       1.20       0.14            -0.08             0.45                 0.92                 1.57
ly %         -0.02       0.98       0.05            -0.12             0.08                 0.89                 1.09
ly (k/ul)    -0.47       0.63       0.59            -1.63             0.70                 0.20                 2.01
mch (pg)      0.33       1.39       0.37            -0.39             1.05                 0.67                 2.86
mchc (g/dl)   0.09       1.09       0.03             0.02             0.15                 1.02                 1.16
mcv(fl)      -0.07       0.93       0.14            -0.34             0.20                 0.71                 1.22
plt (k/ul)    0.00       1.00       0.00             0.00             0.00                 1.00                 1.00
rbc (m/ul)   -0.05       0.95       0.46            -0.95             0.85                 0.39                 2.34
wbc (k/ul)    0.49       1.63       0.58            -0.65             1.62                 0.52                 5.06

              cmp to     z      p   -log2(p)
covariate                                   
sex_int         0.00 -5.88 <0.005      27.82
gr %            0.00  0.35   0.73       0.46
gr (k/ul)       0.00 -0.93   0.35       1.50
hb (g/dl)       0.00 -2.16   0.03       5.03
hct %           0.00  1.35   0.18       2.50
ly %            0.00 -0.36   0.72       0.47
ly (k/ul)       0.00 -0.78   0.43       1.21
mch (pg)        0.00  0.89   0.37       1.43
mchc (g/dl)     0.00  2.67   0.01       7.03
mcv(fl)         0.00 -0.51   0.61       0.71
plt (k/ul)      0.00  3.64 <0.005      11.85
rbc (m/ul)      0.00 -0.11   0.91       0.14
wbc (k/ul)      0.00  0.84   0.40       1.32
---
Concordance = 0.69
Partial AIC = 11203.44
log-likelihood ratio test = 333.10 on 13 df
-log2(p) of ll-ratio test = 207.81

## Calculate spearman’s rank-order test (table 1)

In [4]:
res_spear, latex_spear = plots.table_MPD_survival(df_data_mpd, ['dFI', 'Cox PH'], return_latex=True, fmt='exp')
display(res_spear)
try:
    # res_spear.clear()
    with pd.ExcelWriter('./source/table1_lifespan_mpd.xlsx', engine="openpyxl",  mode="a", if_sheet_exists="replace") as writer:
        res_spear.to_excel(writer)
except FileNotFoundError:
    with pd.ExcelWriter('./source/table1_lifespan_mpd.xlsx', engine="openpyxl") as writer:
        res_spear.to_excel(writer)

N strains males 27, N strains females 29


,M (26 w),F (26 w),M (52 w),F (52 w),M (78 w),F (78 w)
Cohort size 1,91,125,148,188,173,150
dFI (all mice),-0.31 (2.9e-03),-0.18 (4.8e-02),-0.38 (2.2e-06),-0.23 (1.4e-03),-0.37 (7.8e-07),-0.28 (5.2e-04)
Cox PH (all mice),-0.31 (2.5e-03),-0.10 (2.6e-01),-0.41 (3.2e-07),-0.20 (6.4e-03),-0.37 (4.6e-07),-0.35 (1.5e-05)
Cohort size 2,79,118,139,177,133,129
dFI (IGF1 subset),-0.34 (2.4e-03),-0.21 (2.1e-02),-0.37 (6.3e-06),-0.25 (7.8e-04),-0.32 (1.8e-04),-0.31 (3.2e-04)
Cox PH (IGF1 subset),-0.31 (5.9e-03),-0.14 (1.2e-01),-0.40 (9.2e-07),-0.22 (3.8e-03),-0.31 (3.5e-04),-0.39 (6.4e-06)
IGF1,-0.28 (7.7e-03),-0.17 (6.9e-02),-0.12 (1.4e-01),-0.11 (1.2e-01),0.04 (6.9e-01),0.05 (5.6e-01)
Body weight,-0.26 (1.3e-02),-0.19 (3.3e-02),-0.25 (3.0e-03),-0.24 (9.7e-04),0.03 (6.9e-01),0.04 (6.8e-01)


In [5]:
with open('figs/table1_dfi_spear.tex', 'wt') as f:
    f.writelines(latex_spear)

## Calculate Cox PH model and show concordance index (table s6)

In [6]:
res_Cox = plots.table_MPD_survival(df_data_mpd, ['dFI', 'Cox PH'], cox=True, return_latex=False, fmt='exp')
display(res_Cox)
try:
    # res_Cox.clear()
    with pd.ExcelWriter('./source/Supplementary_Data_6.xlsx', engine="openpyxl",  mode="a", if_sheet_exists="replace") as writer:
        res_Cox.to_excel(writer)
except FileNotFoundError:
    with pd.ExcelWriter('./source/Supplementary_Data_6.xlsx', engine="openpyxl") as writer:
        res_Cox.to_excel(writer)


N strains males 27, N strains females 29


,M (26 w),F (26 w),M (52 w),F (52 w),M (78 w),F (78 w)
Cohort size 1,91,125,148,188,173,150
dFI (all mice),"1.37, (1.12‒1.68, p=2.6e-03)","1.25, (1.05‒1.49, p=1.1e-02)","1.72, (1.42‒2.08, p=3.5e-08)","1.29, (1.11‒1.51, p=1.0e-03)","1.53, (1.27‒1.83, p=4.8e-06)","1.44, (1.20‒1.73, p=1.0e-04)"
Cox PH (all mice),"1.45, (1.16‒1.82, p=1.1e-03)","1.21, (1.02‒1.44, p=2.6e-02)","1.70, (1.44‒2.00, p=1.9e-10)","1.39, (1.19‒1.63, p=3.6e-05)","1.71, (1.44‒2.02, p=4.5e-10)","1.71, (1.43‒2.05, p=3.4e-09)"
Cohort size 2,79,118,139,177,133,129
dFI (IGF1 subset),"1.39, (1.12‒1.73, p=3.4e-03)","1.28, (1.07‒1.52, p=7.1e-03)","1.77, (1.44‒2.18, p=4.4e-08)","1.32, (1.14‒1.54, p=3.3e-04)","1.36, (1.12‒1.64, p=1.5e-03)","1.43, (1.18‒1.74, p=2.8e-04)"
Cox PH (IGF1 subset),"1.46, (1.14‒1.86, p=2.5e-03)","1.22, (1.03‒1.45, p=2.1e-02)","1.83, (1.54‒2.19, p=2.1e-11)","1.43, (1.22‒1.68, p=1.4e-05)","1.42, (1.18‒1.70, p=1.4e-04)","1.68, (1.40‒2.03, p=3.9e-08)"
IGF1,"1.33, (1.06‒1.68, p=1.6e-02)","1.27, (1.04‒1.54, p=1.7e-02)","1.12, (0.93‒1.36, p=2.3e-01)","1.18, (1.02‒1.36, p=2.5e-02)","0.99, (0.83‒1.20, p=9.5e-01)","1.12, (0.92‒1.36, p=2.5e-01)"
Body weight,"1.62, (1.23‒2.14, p=6.2e-04)","1.44, (1.20‒1.71, p=5.4e-05)","1.38, (1.20‒1.59, p=9.2e-06)","1.41, (1.23‒1.61, p=8.7e-07)","1.06, (0.88‒1.28, p=5.5e-01)","1.16, (0.97‒1.40, p=1.1e-01)"
